# Serve Docs
This is a utlity to serve the docs locally.

1. Run a CLUE instance locally
2. Configure and run the cell below.
3. A list of links to to the documents as raw json and in document viewer will be visible at this url {{ insert url here }}




In [1]:
import os
import http.server

cwd = os.getcwd()
print(f'Current working directory: {cwd}')
# if the cwd does not end with `docsfiles`, abort script
if not cwd.endswith('docsfiles'):
  print('Please run this script from the `docsfiles` directory')
  exit()

doc_directory = '/documents'  # change this to an existing directory
port = 8081

class MyHTTPRequestHandler(http.server.SimpleHTTPRequestHandler):
  def do_GET(self):
    if self.path == '/documents':
      self.send_response(200)
      self.send_header('Content-type', 'text/html')
      self.end_headers()

      files = os.listdir(doc_directory)
      html = '<html><body><table><tr><th>Link</th><th>Document Name</th></tr>'
      for file in files:
        html += f'<tr><td><a href="/{file}">{file}</a></td><td>{file}</td></tr>'
      html += '</table></body></html>'

      self.wfile.write(html.encode())
    else:
      super().do_GET()

os.chdir(doc_directory)
http.server.test(HandlerClass=MyHTTPRequestHandler, port=port)

Current working directory: /Users/josephbacal/Projects/CC/docsfiles
Serving HTTP on :: port 8081 (http://[::]:8081/) ...


::1 - - [09/Nov/2023 12:12:37] "GET / HTTP/1.1" 200 -
----------------------------------------
Exception occurred during processing of request from ('::1', 53211, 0, 0)
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/socketserver.py", line 683, in process_request_thread
    self.finish_request(request, client_address)
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/socketserver.py", line 360, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/http/server.py", line 668, in __init__
    super().__init__(*args, **kwargs)
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/socketserver.py", line 747, in __init__
    self.handle()
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/http/server.py", line 433, in handle
    self.handle_one_request()
  File "/Librar